# parse out SQL data for documenation
then show the JSON in confluence:
https://bobswift.atlassian.net/wiki/spaces/TBL/pages/131104782/JIRA+Version+List+Using+the+JSON+Table+Macro

# Notes
rules for input sql:

1) all fields need to start with comma (except first row)

In [721]:
import itertools
import json
inputfile = './sql-to-parse.sql'

In [722]:
zipped = {}
n = 0
with open(inputfile, 'r') as f:

    for line in f:
#     for group, lines in itertools.groupby(f, lambda l: l.startswith(",")):
        # print 'line {} {}'.format(group, list(lines))
        zipped[n] = {'concatAbove': line.startswith(","), 'lineItem': [line]}
        n = n + 1
# print(zipped)

for key, value in zipped.items():
    print(key, '->', value)
print ('\n')

0 -> {'concatAbove': False, 'lineItem': ['select\n']}
1 -> {'concatAbove': True, 'lineItem': [',"A11 Order"."order id"\n']}
2 -> {'concatAbove': True, 'lineItem': [',"A11 Work Order"."work order id"\n']}
3 -> {'concatAbove': False, 'lineItem': ['\n']}
4 -> {'concatAbove': True, 'lineItem': [', "all_exclusions" = \'N\'\n']}
5 -> {'concatAbove': False, 'lineItem': ['as "order-tf"\n']}
6 -> {'concatAbove': False, 'lineItem': ['\n']}
7 -> {'concatAbove': True, 'lineItem': [',"order-tf"\n']}
8 -> {'concatAbove': False, 'lineItem': ['and "fault within-tf"\n']}
9 -> {'concatAbove': False, 'lineItem': ['and "snap date" > \'2019-01-01\'\n']}
10 -> {'concatAbove': False, 'lineItem': ['as "metric 1"\n']}
11 -> {'concatAbove': False, 'lineItem': ['\n']}
12 -> {'concatAbove': True, 'lineItem': [', "incident-tf"\n']}
13 -> {'concatAbove': False, 'lineItem': ['as "metric2"\n']}
14 -> {'concatAbove': False, 'lineItem': ['\n']}
15 -> {'concatAbove': False, 'lineItem': ['from cc_dde."order_t" as "A11 Or

## Add rows together and seperate based on sql comma

In [723]:
dict(itertools.islice(zipped.items(),3,5))

{3: {'concatAbove': False, 'lineItem': ['\n']},
 4: {'concatAbove': True, 'lineItem': [', "all_exclusions" = \'N\'\n']}}

In [724]:
dave = (12, {'concatAbove': True, 'lineItem': [', "incident-tf"\n', 'as "metric2"\n', '\n', 'from cc_dde."order_t" as "A11 Order"\n', 'inner join fld_dde."work_order_t" as "A11 Work Order" on "A11 Order"."order_key" = "A11 Work Order"."order_key"']})
"from" in dave[1]['lineItem'][3]

True

In [725]:
for row in zipped:
        for rowsbelow in dict(itertools.islice(zipped.items(),row+1,len(zipped))):

            # break if row below is the start of a new field
            if zipped[rowsbelow]['concatAbove'] == True:
                break
            
            # break if row below not a field
            if 'from' in zipped[rowsbelow]['lineItem'][0]:
                break
            
            # add to the top of rows below
            zipped[row]['lineItem'] = zipped[row]['lineItem'] + zipped[rowsbelow]['lineItem']

# print(json.dumps(zipped,indent=4))
for line in zipped.items():
    print(line)


(0, {'concatAbove': False, 'lineItem': ['select\n']})
(1, {'concatAbove': True, 'lineItem': [',"A11 Order"."order id"\n']})
(2, {'concatAbove': True, 'lineItem': [',"A11 Work Order"."work order id"\n', '\n']})
(3, {'concatAbove': False, 'lineItem': ['\n']})
(4, {'concatAbove': True, 'lineItem': [', "all_exclusions" = \'N\'\n', 'as "order-tf"\n', '\n']})
(5, {'concatAbove': False, 'lineItem': ['as "order-tf"\n', '\n']})
(6, {'concatAbove': False, 'lineItem': ['\n']})
(7, {'concatAbove': True, 'lineItem': [',"order-tf"\n', 'and "fault within-tf"\n', 'and "snap date" > \'2019-01-01\'\n', 'as "metric 1"\n', '\n']})
(8, {'concatAbove': False, 'lineItem': ['and "fault within-tf"\n', 'and "snap date" > \'2019-01-01\'\n', 'as "metric 1"\n', '\n']})
(9, {'concatAbove': False, 'lineItem': ['and "snap date" > \'2019-01-01\'\n', 'as "metric 1"\n', '\n']})
(10, {'concatAbove': False, 'lineItem': ['as "metric 1"\n', '\n']})
(11, {'concatAbove': False, 'lineItem': ['\n']})
(12, {'concatAbove': True, 

In [726]:
zipped

{0: {'concatAbove': False, 'lineItem': ['select\n']},
 1: {'concatAbove': True, 'lineItem': [',"A11 Order"."order id"\n']},
 2: {'concatAbove': True,
  'lineItem': [',"A11 Work Order"."work order id"\n', '\n']},
 3: {'concatAbove': False, 'lineItem': ['\n']},
 4: {'concatAbove': True,
  'lineItem': [', "all_exclusions" = \'N\'\n', 'as "order-tf"\n', '\n']},
 5: {'concatAbove': False, 'lineItem': ['as "order-tf"\n', '\n']},
 6: {'concatAbove': False, 'lineItem': ['\n']},
 7: {'concatAbove': True,
  'lineItem': [',"order-tf"\n',
   'and "fault within-tf"\n',
   'and "snap date" > \'2019-01-01\'\n',
   'as "metric 1"\n',
   '\n']},
 8: {'concatAbove': False,
  'lineItem': ['and "fault within-tf"\n',
   'and "snap date" > \'2019-01-01\'\n',
   'as "metric 1"\n',
   '\n']},
 9: {'concatAbove': False,
  'lineItem': ['and "snap date" > \'2019-01-01\'\n', 'as "metric 1"\n', '\n']},
 10: {'concatAbove': False, 'lineItem': ['as "metric 1"\n', '\n']},
 11: {'concatAbove': False, 'lineItem': ['\n'

In [727]:
newDict = { key:value for (key,value) in zipped.items() if value['concatAbove'] == True}

In [728]:
newDict

{1: {'concatAbove': True, 'lineItem': [',"A11 Order"."order id"\n']},
 2: {'concatAbove': True,
  'lineItem': [',"A11 Work Order"."work order id"\n', '\n']},
 4: {'concatAbove': True,
  'lineItem': [', "all_exclusions" = \'N\'\n', 'as "order-tf"\n', '\n']},
 7: {'concatAbove': True,
  'lineItem': [',"order-tf"\n',
   'and "fault within-tf"\n',
   'and "snap date" > \'2019-01-01\'\n',
   'as "metric 1"\n',
   '\n']},
 12: {'concatAbove': True,
  'lineItem': [', "incident-tf"\n', 'as "metric2"\n', '\n']}}

## remove the first comma from all lineItems

In [729]:
for field in newDict:
    newDict[field]['lineItem'][0] = newDict[field]['lineItem'][0].replace(",","")

## if line has an 'as' then change that to the key

## if key = value then no calc
types = calculation, or reference

structure

[
    {
        columnName: x,
        type: calculation/field,
        calculation: x/null,
        sourceTable: <if field>
    },
    {
        columnName: x,
        type: calculation/field,
        calculation: x/null,
        sourceTable: <if field>
    }
]
    
or

{
    columnName :{
        type: calculation/field,
        calculation: list of strings/null,
        sourceTable: <if field>
    },
    columnName: {
        type: calculation/field,
        calculation: list of strings/null,
        sourceTable: <if field>
    }
}

In [730]:
documenation = {}
for field in newDict.items():
    
    # get column name from line items if as is present
    columnName = ''
    for line in field[1]['lineItem']:
        if 'as ' in line.lower():
            columnName = line[line.find('as ')+3:-1]
#             print(metricName)
            
    # if no rows have a 'as' then must just be a row
    if columnName == '':
        columnName = ''.join(field[1]['lineItem'])
    

    # if columnName = calculation then this isnt a calc! its a field
    if columnName == ''.join(field[1]['lineItem']):
        
        # split out table alias and the actual field
        if '.' in ''.join(field[1]['lineItem']):
            tableAlias, fieldTableRemoved = ''.join(field[1]['lineItem']).split('.')
            documenation[fieldTableRemoved] = {
                'type': 'field',
                'tableAlias': tableAlias
            }
        else:
            documenation[columnName] = {
                'type': 'field'
            }
    
    # else then add the calculation
    else:
        documenation[columnName] = {
            'calculation': newDict[field[0]]['lineItem'],
            'type': 'calc'
                                   }

print('\n')
for item in documenation.items():
    print(item)



('"order id"\n', {'type': 'field', 'tableAlias': '"A11 Order"'})
('"work order id"\n\n', {'type': 'field', 'tableAlias': '"A11 Work Order"'})
('"order-tf"', {'calculation': [' "all_exclusions" = \'N\'\n', 'as "order-tf"\n', '\n'], 'type': 'calc'})
('"metric 1"', {'calculation': ['"order-tf"\n', 'and "fault within-tf"\n', 'and "snap date" > \'2019-01-01\'\n', 'as "metric 1"\n', '\n'], 'type': 'calc'})
('"metric2"', {'calculation': [' "incident-tf"\n', 'as "metric2"\n', '\n'], 'type': 'calc'})


## clean up

### remove filed name

In [731]:
for key in documenation:
    if 'calculation' in documenation[key]:
        updated_calculation = []
        for line in documenation[key]['calculation']:
            if key not in line:
                updated_calculation.append(line)
        documenation[key]['calculation'] = updated_calculation

for item in documenation.items():
    print(item)
            

('"order id"\n', {'type': 'field', 'tableAlias': '"A11 Order"'})
('"work order id"\n\n', {'type': 'field', 'tableAlias': '"A11 Work Order"'})
('"order-tf"', {'calculation': [' "all_exclusions" = \'N\'\n', '\n'], 'type': 'calc'})
('"metric 1"', {'calculation': ['"order-tf"\n', 'and "fault within-tf"\n', 'and "snap date" > \'2019-01-01\'\n', '\n'], 'type': 'calc'})
('"metric2"', {'calculation': [' "incident-tf"\n', '\n'], 'type': 'calc'})


# recursive logic to replace fields

# try printing out
need to deal with new lines and escape chars

In [732]:
documenation

{'"order id"\n': {'type': 'field', 'tableAlias': '"A11 Order"'},
 '"work order id"\n\n': {'type': 'field', 'tableAlias': '"A11 Work Order"'},
 '"order-tf"': {'calculation': [' "all_exclusions" = \'N\'\n', '\n'],
  'type': 'calc'},
 '"metric 1"': {'calculation': ['"order-tf"\n',
   'and "fault within-tf"\n',
   'and "snap date" > \'2019-01-01\'\n',
   '\n'],
  'type': 'calc'},
 '"metric2"': {'calculation': [' "incident-tf"\n', '\n'], 'type': 'calc'}}

In [733]:
for key in documenation.keys():
    print("KEY = {}".format(key))
    if 'calculation' in documenation[key]:
        print("CALCULATION = {",)

        for multi_line in documenation[key]['calculation']:
            print(multi_line)
        print("}",)

KEY = "order id"

KEY = "work order id"


KEY = "order-tf"
CALCULATION = {
 "all_exclusions" = 'N'



}
KEY = "metric 1"
CALCULATION = {
"order-tf"

and "fault within-tf"

and "snap date" > '2019-01-01'



}
KEY = "metric2"
CALCULATION = {
 "incident-tf"



}
